## Drafts on t tests and power analyses

The power analysis is at the bottom

In [309]:
# most of these arent used here

#import mne 
import numpy as np
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import sys
import warnings
import seaborn as sns
from itertools import product as prod

# temporary (used only here)
from scipy import stats
from statsmodels.stats.power import ttest_power, tt_ind_solve_power



Doing t-tests on arrays, if we wanted to do this at some point

In [314]:
# create fake arrays of SEs, using 4 electrodes
arr_ele = np.array([.01,1,5,10])

# fake subjects and effect sizes
Subs = [15,20,25,30]
effect = [1,2,3,5]


In [315]:
# t tests with simple array

t_arr = np.zeros((len(arr_ele), len(Subs), len(effect))) 
p_arr = np.zeros((len(arr_ele), len(Subs), len(effect)))

for ele in range(len(arr_ele)):  # loop over electrodes
    BESE = arr_simp[ele]
    
    for s in range(len(Subs)):    # loop over sample sizes
        n = Subs[s]
        df = 2*n - 2              # get degrees of freedom (based on sample size)
        
        for ef in range(len(effect)):          # loop over effect sizes
            t = effect[ef] / (BESE/np.sqrt(n)) # get t value 
            t_arr[ele][s][ef] = t              # save t value
            
            p = 1 - stats.t.cdf(t,df=df)       # get p value
            p_arr[ele][s][ef] = p              # save p value


In [316]:
# t values
print(t_arr)
# p values
print(p_arr)

[[[ 3.87298335  7.74596669 11.61895004 19.36491673]
  [ 4.47213595  8.94427191 13.41640786 22.36067977]
  [ 5.         10.         15.         25.        ]
  [ 5.47722558 10.95445115 16.43167673 27.38612788]]

 [[ 1.93649167  3.87298335  5.80947502  9.68245837]
  [ 2.23606798  4.47213595  6.70820393 11.18033989]
  [ 2.5         5.          7.5        12.5       ]
  [ 2.73861279  5.47722558  8.21583836 13.69306394]]

 [[ 1.29099445  2.5819889   3.87298335  6.45497224]
  [ 1.49071198  2.98142397  4.47213595  7.45355992]
  [ 1.66666667  3.33333333  5.          8.33333333]
  [ 1.82574186  3.65148372  5.47722558  9.12870929]]

 [[ 0.77459667  1.54919334  2.32379001  3.87298335]
  [ 0.89442719  1.78885438  2.68328157  4.47213595]
  [ 1.          2.          3.          5.        ]
  [ 1.09544512  2.19089023  3.28633535  5.47722558]]]
[[[2.95046492e-04 9.71734049e-09 1.58861813e-12 0.00000000e+00]
  [3.39951142e-05 3.44879680e-11 3.33066907e-16 0.00000000e+00]
  [4.03083259e-06 1.26676447e-13

### Power analysis on 1d arrays

In [317]:
# fake arrays

Subs = [1,10,20,50]
effects = [1, 2, 3, 5]
arr_ele = np.array([.1,2,3,10])


In [318]:
pwrs = np.zeros((len(arr_ele), len(Subs), len(effects)))

for ele in range(len(arr_ele)):  # loop over electrodes
    BESE = arr_simp[ele]
    
    for s in range(len(Subs)):    # loop over sample sizes
        n = Subs[s]
        SD = BESE/(np.sqrt(n))    # get SD from BESE (maybe wrong)
        #null_dist = np.random.normal(loc = 0, scale = SD, size = 1000)     # make null distribution
        crit = 1.96*SD                                                      # get critical value
        
        for effect in range(len(effects)):                                  # loop over effect sizes
            ef = effects[effect]
            alt_dist = np.random.normal(loc = ef, scale = SD, size = 10000) # make alternative distribution 
            pwr = len(alt_dist[alt_dist>crit])/len(alt_dist)
            
            pwrs[ele][s][effect] = pwr              # save power 


In [319]:
pwrs

array([[[0.1689, 0.5104, 0.8495, 0.9983],
        [0.887 , 1.    , 1.    , 1.    ],
        [0.9941, 1.    , 1.    , 1.    ],
        [1.    , 1.    , 1.    , 1.    ]],

       [[0.0712, 0.171 , 0.3198, 0.7061],
        [0.349 , 0.8848, 0.9978, 1.    ],
        [0.6073, 0.9945, 1.    , 1.    ],
        [0.9456, 1.    , 1.    , 1.    ]],

       [[0.0533, 0.0949, 0.1598, 0.3842],
        [0.1907, 0.5552, 0.8833, 0.9996],
        [0.3255, 0.8411, 0.9943, 1.    ],
        [0.6525, 0.9975, 1.    , 1.    ]],

       [[0.0363, 0.0652, 0.0878, 0.171 ],
        [0.0895, 0.2464, 0.475 , 0.8795],
        [0.1405, 0.4405, 0.7658, 0.9941],
        [0.3001, 0.8058, 0.9883, 1.    ]]])

### Power analysis with 2d array input (for item option)

In [218]:
# now for the item option...

# here, we will have to make effect size be a constant

effect = 2
Subs = [1,10] # usually would be 4

# sample array
arr_item = np.array([[1,2,3,4],[2,4,6,8],[3,6,9,12]]) # rows are items(usually would be 4), columns are electrodes
arr_item

array([[ 1,  2,  3,  4],
       [ 2,  4,  6,  8],
       [ 3,  6,  9, 12]])

In [219]:
pwrs_i = np.zeros((arr_item.shape[1], arr_item.shape[0], len(Subs)))

for ele in range(arr_item.shape[1]):  # loop over electrodes
    
    for item in range(arr_item.shape[0]):    # loop over item samples
        BESE = arr_item[item][ele]
          
        for s in range(len(Subs)):               # loop over subject samples
            n = Subs[s]
            SD = BESE/(np.sqrt(n))                                              # get SD from BESE (maybe wrong)
            crit = 1.96*SD                                                      # get critical value
            alt_dist = np.random.normal(loc = effect, scale = SD, size = 10000) # make alternative distribution 
            pwr_i = len(alt_dist[alt_dist>crit])/len(alt_dist)                  # get power
            
            pwrs_i[ele][item][s] = pwr_i                                        # store powers

In [220]:
pwrs_i # z = electrodes, y = subs, x = item samples

array([[[0.5169, 1.    ],
        [0.1642, 0.8839],
        [0.0959, 0.5561]],

       [[0.1641, 0.8805],
        [0.0694, 0.3477],
        [0.0521, 0.1753]],

       [[0.0964, 0.5646],
        [0.0514, 0.1849],
        [0.0409, 0.1019]],

       [[0.0703, 0.3644],
        [0.0443, 0.128 ],
        [0.0343, 0.0736]]])

# The two combined in function

Not the final committed version
(the one in eeg_pwr doesnt have item analysis)

Does normal power analysis if the input BESE array is 1d;
does the item version if the input BESE array is 2d

In [331]:
def eeg_pwr(SE_array , Subs , Effects):
    
    if len(SE_array.shape) == 1:           # Array of electrode BESEs
        
        pwrs = np.zeros((len(SE_array), len(Subs), len(Effects)))  

        for ele in range(len(SE_array)):  # loop over electrodes
            BESE = arr_simp[ele]

            for s in range(len(Subs)):    # loop over sample sizes
                n = Subs[s]
                SD = BESE/(np.sqrt(n))    # get SD from BESE (maybe wrong)
                crit = 1.96*SD            # get critical value

                for effect in range(len(Effects)):                                  # loop over effect sizes
                    ef = Effects[effect]
                    alt_dist = np.random.normal(loc = ef, scale = SD, size = 10000) # make alternative distribution 
                    pwr = len(alt_dist[alt_dist>crit])/len(alt_dist)

                    pwrs[ele][s][effect] = pwr              # save power
                    
        return pwrs
        
    if len(SE_array.shape) == 2:          # Array of electrode BESEs for different numbers of items used in bootstrap
        
        pwrs_i = np.zeros((SE_array.shape[1], SE_array.shape[0], len(Subs)))

        for ele in range(SE_array.shape[1]):         # loop over electrodes

            for item in range(SE_array.shape[0]):    # loop over item samples
                BESE = SE_array[item][ele]

                for s in range(len(Subs)):               # loop over subject samples
                    n = Subs[s]
                    SD = BESE/(np.sqrt(n))                                              # get SD from BESE (maybe wrong)
                    crit = 1.96*SD                                                      # get critical value
                    alt_dist = np.random.normal(loc = Effects, scale = SD, size = 10000)# make alternative distribution 
                    pwr_i = len(alt_dist[alt_dist>crit])/len(alt_dist)                  # get power

                    pwrs_i[ele][item][s] = pwr_i                                        # store powers
                    
        return pwrs_i
    
    else:
        print('Function requires 1 or 2 dimension array')

Testing....

In [332]:
#test 1d
effects = [2,3,4]
subs = [1,10] # usually would be 4
array_simp = np.array([1,2,3,4])

test_1 = eeg_pwr(SE_array = array_simp, Effects = effects, Subs = subs)
test_1

array([[[0.5154, 0.8498, 0.9757],
        [0.9999, 1.    , 1.    ]],

       [[0.1669, 0.3213, 0.5179],
        [0.889 , 0.9973, 1.    ]],

       [[0.0945, 0.1743, 0.2631],
        [0.566 , 0.8896, 0.9877]],

       [[0.0623, 0.087 , 0.1202],
        [0.2456, 0.4737, 0.7152]]])

In [330]:
#test 2d
effect = [2] # now just 1
subs = [1,10] # usually would be 4
array_2 = np.array([[.1,.2,.3,.4],[2,4,6,8],[3,6,9,12]]) 

test_2 = eeg_pwr(SE_array = array_2, Effects = effect, Subs = subs)
test_2


array([[[1.    , 1.    ],
        [0.1628, 0.8906],
        [0.0933, 0.5629]],

       [[1.    , 1.    ],
        [0.0728, 0.3583],
        [0.0484, 0.1807]],

       [[1.    , 1.    ],
        [0.0507, 0.1802],
        [0.0381, 0.1064]],

       [[0.999 , 1.    ],
        [0.0458, 0.1225],
        [0.035 , 0.0745]]])